In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
df=pd.read_csv("C://Users//Lenovo//Downloads//Assignment//Random Forest//Fraud_check.csv")
df.head()

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [4]:
df=pd.get_dummies(df,columns=['Undergrad','Marital.Status','Urban'], drop_first=True)

In [5]:
df.head()

,Taxable.Income,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,68833,50047,10,0,0,1,1
1,33700,134075,18,1,0,0,1
2,36925,160205,30,0,1,0,1
3,50190,193264,15,1,0,1,1
4,81002,27533,28,0,1,0,0


In [6]:
df["TaxInc"] = pd.cut(df["Taxable.Income"], bins = [10002,30000,99620], labels = ["Risky", "Good"])

In [7]:
df = pd.get_dummies(df,columns = ["TaxInc"],drop_first=True)

In [8]:
df.head()

,Taxable.Income,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES,TaxInc_Good
0,68833,50047,10,0,0,1,1,1
1,33700,134075,18,1,0,0,1,1
2,36925,160205,30,0,1,0,1,1
3,50190,193264,15,1,0,1,1,1
4,81002,27533,28,0,1,0,0,1


In [9]:
df.corr

<bound method DataFrame.corr of      Taxable.Income  City.Population  Work.Experience  Undergrad_YES  \
0             68833            50047               10              0   
1             33700           134075               18              1   
2             36925           160205               30              0   
3             50190           193264               15              1   
4             81002            27533               28              0   
..              ...              ...              ...            ...   
595           76340            39492                7              1   
596           69967            55369                2              1   
597           47334           154058                0              0   
598           98592           180083               17              1   
599           96519           158137               16              0   

     Marital.Status_Married  Marital.Status_Single  Urban_YES  TaxInc_Good  
0                         

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [11]:
X = df.iloc[:,1:]
calc_vif(X)

,variables,VIF
0,City.Population,4.221463
1,Work.Experience,3.346069
2,Undergrad_YES,2.015139
3,Marital.Status_Married,1.860291
4,Marital.Status_Single,1.977209
5,Urban_YES,1.878578
6,TaxInc_Good,3.672330


In [12]:
df=df.drop('City.Population',axis=1)

In [13]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [14]:
df_norm = norm_func(df.iloc[:,1:])
df_norm.tail(10)

,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES,TaxInc_Good
590,0.466667,0.0,1.0,0.0,1.0,1.0
591,0.600000,1.0,0.0,1.0,1.0,0.0
592,0.533333,1.0,0.0,1.0,0.0,1.0
593,0.333333,0.0,0.0,0.0,1.0,1.0
594,0.300000,0.0,1.0,0.0,1.0,1.0
595,0.233333,1.0,0.0,0.0,1.0,1.0
596,0.066667,1.0,0.0,0.0,1.0,1.0
597,0.000000,0.0,0.0,0.0,1.0,1.0
598,0.566667,1.0,1.0,0.0,0.0,1.0
599,0.533333,0.0,0.0,0.0,0.0,1.0


In [15]:
X = df_norm.drop(['TaxInc_Good'], axis=1)
y = df_norm['TaxInc_Good']

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7,shuffle=True)

In [18]:
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(x_train,y_train)

RandomForestClassifier(max_features=3)

In [19]:
results = cross_val_score(model, X, y, cv=kfold)

In [20]:
results

array([0.68333333, 0.7       , 0.73333333, 0.7       , 0.71666667,
       0.68333333, 0.71666667, 0.56666667, 0.73333333, 0.71666667])

In [21]:
print(results.mean())

0.6950000000000001


In [22]:
preds=model.predict(x_test)
pd.Series(preds).value_counts()

1.0    111
0.0      9
dtype: int64

In [23]:
preds

array([0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1.])

In [24]:
pd.crosstab(y_test,preds)

col_0,0.0,1.0
TaxInc_Good,,
0.0,2,23
1.0,7,88


In [25]:
np.mean(preds==y_test)

0.75

# Company_check

In [42]:
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [45]:
df=pd.read_csv("C://Users//Lenovo//Downloads//Assignment//Random Forest//Company_Data.csv")
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [47]:
df=pd.get_dummies(df,columns=['ShelveLoc','US','Urban'], drop_first=True)
df

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,US_Yes,Urban_Yes
0,9.50,138,73,11,276,120,42,17,0,0,1,1
1,11.22,111,48,16,260,83,65,10,1,0,1,1
2,10.06,113,35,10,269,80,59,12,0,1,1,1
3,7.40,117,100,4,466,97,55,14,0,1,1,1
4,4.15,141,64,3,340,128,38,13,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,33,14,1,0,1,1
396,6.14,139,23,3,37,120,55,11,0,1,1,0
397,7.41,162,26,12,368,159,40,18,0,1,1,1
398,5.94,100,79,7,284,95,50,12,0,0,1,1


In [48]:
df["Sale"] = pd.cut(df["Sales"], bins = [2,9,15], labels = ["No", "Yes"])
df

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,US_Yes,Urban_Yes,Sale
0,9.50,138,73,11,276,120,42,17,0,0,1,1,Yes
1,11.22,111,48,16,260,83,65,10,1,0,1,1,Yes
2,10.06,113,35,10,269,80,59,12,0,1,1,1,Yes
3,7.40,117,100,4,466,97,55,14,0,1,1,1,No
4,4.15,141,64,3,340,128,38,13,0,0,0,1,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,33,14,1,0,1,1,Yes
396,6.14,139,23,3,37,120,55,11,0,1,1,0,No
397,7.41,162,26,12,368,159,40,18,0,1,1,1,No
398,5.94,100,79,7,284,95,50,12,0,0,1,1,No


In [49]:
df = pd.get_dummies(df,columns = ["Sale"],drop_first=True)
df.head(15)

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,US_Yes,Urban_Yes,Sale_Yes
0,9.50,138,73,11,276,120,42,17,0,0,1,1,1
1,11.22,111,48,16,260,83,65,10,1,0,1,1,1
2,10.06,113,35,10,269,80,59,12,0,1,1,1,1
3,7.40,117,100,4,466,97,55,14,0,1,1,1,0
4,4.15,141,64,3,340,128,38,13,0,0,0,1,0
5,10.81,124,113,13,501,72,78,16,0,0,1,0,1
6,6.63,115,105,0,45,108,71,15,0,1,0,1,0
7,11.85,136,81,15,425,120,67,10,1,0,1,1,1
8,6.54,132,110,0,108,124,76,10,0,1,0,0,0
9,4.69,132,113,0,131,124,76,17,0,1,1,0,0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sales             400 non-null    float64
 1   CompPrice         400 non-null    int64  
 2   Income            400 non-null    int64  
 3   Advertising       400 non-null    int64  
 4   Population        400 non-null    int64  
 5   Price             400 non-null    int64  
 6   Age               400 non-null    int64  
 7   Education         400 non-null    int64  
 8   ShelveLoc_Good    400 non-null    uint8  
 9   ShelveLoc_Medium  400 non-null    uint8  
 10  US_Yes            400 non-null    uint8  
 11  Urban_Yes         400 non-null    uint8  
 12  Sale_Yes          400 non-null    uint8  
dtypes: float64(1), int64(7), uint8(5)
memory usage: 27.1 KB


In [51]:
df[df.duplicated()]

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,US_Yes,Urban_Yes,Sale_Yes


In [52]:
df.isnull().sum()

Sales               0
CompPrice           0
Income              0
Advertising         0
Population          0
Price               0
Age                 0
Education           0
ShelveLoc_Good      0
ShelveLoc_Medium    0
US_Yes              0
Urban_Yes           0
Sale_Yes            0
dtype: int64

In [53]:
df.corr()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,US_Yes,Urban_Yes,Sale_Yes
Sales,1.000000,0.064079,0.151951,0.269507,0.050471,-0.444951,-0.231815,-0.051955,0.500510,-0.073999,0.177093,-0.015419,0.741904
CompPrice,0.064079,1.000000,-0.080653,-0.024199,-0.094707,0.584848,-0.100239,0.025197,0.026386,0.008683,0.016869,0.066594,0.010126
Income,0.151951,-0.080653,1.000000,0.058995,-0.007877,-0.056698,-0.004670,-0.056855,-0.012657,-0.051392,0.089601,0.037967,0.137233
Advertising,0.269507,-0.024199,0.058995,1.000000,0.265652,0.044537,-0.004557,-0.033594,0.056149,-0.015929,0.684460,0.042035,0.279511
Population,0.050471,-0.094707,-0.007877,0.265652,1.000000,-0.012144,-0.042663,-0.106378,0.007789,-0.040640,0.060564,-0.052025,0.049682
Price,-0.444951,0.584848,-0.056698,0.044537,-0.012144,1.000000,-0.102177,0.011747,0.045854,-0.006607,0.057861,0.047016,-0.324944
Age,-0.231815,-0.100239,-0.004670,-0.004557,-0.042663,-0.102177,1.000000,0.006488,-0.022818,0.056613,0.008652,0.028479,-0.174192
Education,-0.051955,0.025197,-0.056855,-0.033594,-0.106378,0.011747,0.006488,1.000000,-0.029188,0.013242,-0.078250,-0.033094,-0.046718
ShelveLoc_Good,0.500510,0.026386,-0.012657,0.056149,0.007789,0.045854,-0.022818,-0.029188,1.000000,-0.571396,0.078864,-0.039197,0.442382
ShelveLoc_Medium,-0.073999,0.008683,-0.051392,-0.015929,-0.040640,-0.006607,0.056613,0.013242,-0.571396,1.000000,-0.065656,-0.037391,-0.154490


In [54]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [55]:
X = df.iloc[:,1:]
calc_vif(X)

,variables,VIF
0,CompPrice,85.308630
1,Income,6.860377
2,Advertising,4.469242
3,Population,4.420311
4,Price,54.865502
5,Age,11.448012
6,Education,22.898130
7,ShelveLoc_Good,2.673652
8,ShelveLoc_Medium,3.419224
9,US_Yes,5.518990


In [56]:
df1=df.drop('CompPrice',axis=1)
df2=df1.drop('Price',axis=1)
df3=df2.drop('Education',axis=1)
df4=df3.drop('Age',axis=1)

In [57]:
df=df4.drop('Income',axis=1)

In [58]:
df['Sales'] = df['Sales'].astype('category')

In [59]:
# Normalization function 
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [60]:
df_norm = norm_func(df.iloc[:,1:])
df_norm.head(10)

,Advertising,Population,ShelveLoc_Good,ShelveLoc_Medium,US_Yes,Urban_Yes,Sale_Yes
0,0.379310,0.533066,0.0,0.0,1.0,1.0,1.0
1,0.551724,0.501002,1.0,0.0,1.0,1.0,1.0
2,0.344828,0.519038,0.0,1.0,1.0,1.0,1.0
3,0.137931,0.913828,0.0,1.0,1.0,1.0,0.0
4,0.103448,0.661323,0.0,0.0,0.0,1.0,0.0
5,0.448276,0.983968,0.0,0.0,1.0,0.0,1.0
6,0.000000,0.070140,0.0,1.0,0.0,1.0,0.0
7,0.517241,0.831663,1.0,0.0,1.0,1.0,1.0
8,0.000000,0.196393,0.0,1.0,0.0,0.0,0.0
9,0.000000,0.242485,0.0,1.0,1.0,0.0,0.0


In [61]:
# Declaring features & target
X = df_norm.drop(['Sale_Yes'], axis=1)
y = df_norm['Sale_Yes']

In [62]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [63]:
x_train.isnull().sum()

Advertising         0
Population          0
ShelveLoc_Good      0
ShelveLoc_Medium    0
US_Yes              0
Urban_Yes           0
dtype: int64

In [64]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7)

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [65]:
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(x_train,y_train)

RandomForestClassifier(max_features=3)

In [66]:
results = cross_val_score(model, X, y, cv=kfold)

In [67]:
print(results.mean())

0.7575000000000001


In [69]:
preds = model.predict(x_test) # predicting on test data set 
pd.Series(preds).value_counts() # getting the count of each category


0.0    60
1.0    20
dtype: int64

In [70]:
preds

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.])

In [71]:
pd.crosstab(y_test,preds)

col_0,0.0,1.0
Sale_Yes,,
0.0,47,6
1.0,13,14


In [72]:
np.mean(preds==y_test)

0.7625